In [2]:
import pandas as pd

Import all the files

In [3]:
visits = pd.read_csv('visits.csv',
                     parse_dates=[1])
cart = pd.read_csv('cart.csv',
                   parse_dates=[1])
                   
checkout = pd.read_csv('checkout.csv',
                       parse_dates=[1])
purchase = pd.read_csv('purchase.csv',
                       parse_dates=[1])

Step 1: Inspect the DataFrames using `print` and `head`

In [4]:
print(visits.head(5))
print(cart.head(5))
print(checkout.head(5))
print(purchase.head(5))

                                user_id          visit_time
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00
                                user_id           cart_time
0  2be90e7c-9cca-44e0-bcc5-124b945ff168 2017-11-07 20:45:00
1  4397f73f-1da3-4ab3-91af-762792e25973 2017-05-27 01:35:00
2  a9db3d4b-0a0a-4398-a55a-ebb2c7adf663 2017-03-04 10:38:00
3  b594862a-36c5-47d5-b818-6e9512b939b3 2017-09-27 08:22:00
4  a68a16e2-94f0-4ce8-8ce3-784af0bbb974 2017-07-26 15:48:00
                                user_id       checkout_time
0  d33bdc47-4afa-45bc-b4e4-dbe948e34c0d 2017-06-25 09:29:00
1  4ac186f0-9954-4fea-8a27-c081e428e34e 2017-04-07 20:11:00
2  3c9c78a7-124a-4b77-8d2e-e1926e011e7d 2017-07-13 11:38:00
3  89fe330a-8966-4756-8f7c-3bdbcd47279a 

Step 2: Left merging visits and cart

In [5]:
visits_cart_left = pd.merge(visits, cart, how='left')
print(visits_cart_left.head(5))

                                user_id          visit_time  \
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00   
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00   
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00   
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00   
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00   

            cart_time  
0                 NaT  
1 2017-01-26 14:44:00  
2 2017-08-20 08:31:00  
3                 NaT  
4                 NaT  


Step 3: How long is `visits_cart`?

In [7]:
length_visits_cart = visits_cart_left.count()
print(length_visits_cart)

user_id       2000
visit_time    2000
cart_time      348
dtype: int64


Step 4: How many timestamps are null for `cart_time`?

In [8]:
cart_time_nulls = visits_cart_left[visits_cart_left.cart_time.isnull()]
print(len(cart_time_nulls))

1652


Step 5: What percentage only visited?

In [9]:
perc_visit_only = len(cart_time_nulls) / len(visits_cart_left) 
print(perc_visit_only)

0.826


Step 6: What percentage placed a t-shirt in their cart but did not checkout?

In [10]:
cart_checkout_left = pd.merge(cart, checkout, how='left')
checkout_nulls = cart_checkout_left[cart_checkout_left.checkout_time.isnull()]
perc_cart_no_checkout = len(checkout_nulls) / len(cart_checkout_left)
print(perc_cart_no_checkout)


0.3505747126436782


Step 7: Merge it all together

In [13]:
all_data = visits.merge(cart, how='left').merge(checkout, how='left').merge(purchase, how='left')
print(all_data.head(10))


                                user_id          visit_time  \
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00   
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00   
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00   
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00   
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00   
5  7bc62113-d51d-4e6b-85e0-1cf29e3de74a 2017-06-03 20:05:00   
6  6f22cd44-bc63-4449-a9af-2651859da2f9 2017-03-18 23:09:00   
7  49a6399b-c095-4e42-92eb-af90abe2bb6a 2017-01-10 12:09:00   
8  653c095d-fd74-40dd-ae3b-5a69a2613629 2017-06-27 19:21:00   
9  3d9fdac1-aae3-4a01-b448-934d12047468 2017-09-24 06:43:00   

            cart_time       checkout_time       purchase_time  
0                 NaT                 NaT                 NaT  
1 2017-01-26 14:44:00 2017-01-26 14:54:00 2017-01-26 15:08:00  
2 2017-08-20 08:31:00                 NaT                 NaT  
3                 NaT                 NaT         

Step 8: % of users who got to checkout but did not purchase

In [15]:
purchase_nulls = all_data[all_data.purchase_time.isnull()]
perc_checkout_no_purchase = len(purchase_nulls) / len(checkout) 
print(perc_checkout_no_purchase)

8.212389380530974


Step 9: check each part of the funnel, let's print all 3 of them again

In [30]:
n_visit = len(all_data)
n_visit_to_cart = len(all_data[all_data.cart_time.notnull()])
n_cart_to_checkout = len(all_data[all_data.checkout_time.notnull()])
n_checkout_to_purchase = len(all_data[all_data.purchase_time.notnull()])

print('The number of user visits is: ' + str(n_visit))
print('The number of users adding an item to their cart is: ' + str(n_visit_to_cart))
print('The number of users going to checkout is: ' + str(n_cart_to_checkout))
print('The number of users making a purchase is: ' + str(n_checkout_to_purchase))

perc_visit_to_cart = n_visit_to_cart / n_visit * 100
perc_cart_to_checkout = n_cart_to_checkout / n_visit_to_cart * 100
perc_checkout_to_purchase = n_checkout_to_purchase / n_cart_to_checkout * 100 

print('The percentage of users visiting the website who put an item in their cart is: ' + str(perc_visit_to_cart))
print('The percentage of users going to checkout after putting an item in their cart is: ' + str(perc_cart_to_checkout))
print('The percentage of users purchasing the item after going to checkout is: ' + str(perc_checkout_to_purchase))

The number of user visits is: 2108
The number of users adding an item to their cart is: 456
The number of users going to checkout is: 334
The number of users making a purchase is: 252
The percentage of users visiting the website who put an item in their cart is: 21.631878557874764
The percentage of users going to checkout after putting an item in their cart is: 73.24561403508771
The percentage of users purchasing the item after going to checkout is: 75.44910179640718


*The weakest part of the funnel is clearly getting a person who visited the site to add a tshirt to their cart. Once they've added a t-shirt to their cart it is fairly likely they end up purchasing it. A suggestion could be to make the add-to-cart button more prominent on the front page.*


Step 10: adding new column

In [32]:
all_data['time_between_visit_to_purchase'] = all_data.purchase_time - all_data.visit_time

Step 11: examine the results

In [33]:
print(all_data.head(10))

                                user_id          visit_time  \
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00   
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00   
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00   
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00   
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00   
5  7bc62113-d51d-4e6b-85e0-1cf29e3de74a 2017-06-03 20:05:00   
6  6f22cd44-bc63-4449-a9af-2651859da2f9 2017-03-18 23:09:00   
7  49a6399b-c095-4e42-92eb-af90abe2bb6a 2017-01-10 12:09:00   
8  653c095d-fd74-40dd-ae3b-5a69a2613629 2017-06-27 19:21:00   
9  3d9fdac1-aae3-4a01-b448-934d12047468 2017-09-24 06:43:00   

            cart_time       checkout_time       purchase_time  \
0                 NaT                 NaT                 NaT   
1 2017-01-26 14:44:00 2017-01-26 14:54:00 2017-01-26 15:08:00   
2 2017-08-20 08:31:00                 NaT                 NaT   
3                 NaT                 NaT     

Step 12: average time to purchase

In [34]:
average_time_to_purchase = all_data.time_between_visit_to_purchase.mean()

In [35]:
print('The average time from visit to purchase is: ' + str(average_time_to_purchase))

The average time from visit to purchase is: 0 days 00:43:12.380952380
